<a href="https://colab.research.google.com/github/Jgallear/CSSP_brazil_23_24/blob/main/cross_validate_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [81]:
!pip install scikeras

In [82]:
!pip install keras==2.12.0

In [83]:

from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import mean_absolute_error
from scipy.stats import pearsonr
#from rasterio import CRS
import functools
import pickle
#import cartopy.feature as cf
from scipy.stats import pearsonr
from sklearn.preprocessing import StandardScaler
#from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering
import seaborn as sns
from sklearn.metrics import mean_absolute_error
from scipy.stats import pearsonr
from sklearn.metrics import r2_score
import itertools
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GroupKFold
from keras.optimizers import RMSprop, Adam, SGD
#from sklearn.model_selection import RandomizedSearchCV
#from scikeras.wrappers import KerasRegressor
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

In [84]:
# to determine split, take random set of 5 years, create evaluation period, then split training in half, and test number of estimators on each half,
# this will mean model is trained number of folds * number of estimators times. (2*6) times

In [85]:
# do cv split based on year, split years into 2 folds and evaluate a range of n_estimators from 2, 10, 50, 100, 200, 500

In [86]:
# load df
df = pd.read_csv('/content/drive/MyDrive/VHI_spei_rzsm_dataset.csv')
print(df)

           lon     lat      vhi-1   precip-1      rzsm-1  spei1-1m  spei2-1m  \
0      -53.375 -32.125  70.414172  259.20056   98.021910  2.192168  1.313241   
1      -53.375 -32.125  79.323750  146.58607   98.162980  1.228278  2.453098   
2      -53.375 -32.125  77.111895  184.82845   97.594270  1.472914  1.588110   
3      -53.375 -32.125  64.150561  108.86273   97.605330  1.680747  1.751371   
4      -53.375 -32.125  54.270359  145.90760  100.000000  1.641170  1.900034   
...        ...     ...        ...        ...         ...       ...       ...   
577682 -47.625  -0.875  41.102345  170.38158   66.929490  0.324701  0.572972   
577683 -47.625  -0.875  49.922513  294.35620   80.138200  1.130223  0.881931   
577684 -47.625  -0.875  52.101102  125.48111   88.888840  2.664571  2.250521   
577685 -47.625  -0.875  46.753572   24.13338   77.834076  2.523025  3.002636   
577686 -47.625  -0.875  48.627606   71.58264   69.860950  1.286047  2.275886   

        spei3-1m   spi01-1   spi02-1   

In [87]:
# need to loop through df separate specified years into train and test
n_estimators_list = [2,10,25,50,100,200,500]
# obtain random set of years for testing
print(df['year'].unique())
test_yrs = np.random.choice(df['year'].unique(),replace=False,size=5)
print(test_yrs)

[2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016
 2017 2018 2019 2020 2021]
[2015 2014 2007 2008 2009]


In [88]:
test_yrs = [2016, 2011, 2019, 2015, 2006]

In [89]:
# remove test_yrs from dataset, then split training into fold1 and fold2
test_df = df[df['year'].isin(test_yrs)]
###
train_df = df[~df['year'].isin(test_yrs)]
# split train_df in half for fold 1 and fold 2
print(len(train_df['year'].unique()))
print(int(len(train_df['year'].unique())/2))
split = int(len(train_df['year'].unique())/2)
print(train_df['year'].unique())
f1_years = train_df['year'].unique()[:split]
print(f1_years)
f2_years = train_df['year'].unique()[split:]
print(f2_years)
## use f1 years and f2 years to split dataframe
train_df_ri = train_df.reset_index()
f1_df = train_df_ri[~train_df_ri['year'].isin(f1_years)]
f2_df = train_df_ri[~train_df_ri['year'].isin(f2_years)]

f1_inds = f1_df.index
f2_inds = f2_df.index

14
7
[2003 2004 2005 2007 2008 2009 2010 2012 2013 2014 2017 2018 2020 2021]
[2003 2004 2005 2007 2008 2009 2010]
[2012 2013 2014 2017 2018 2020 2021]


In [90]:
print(max(f1_inds))
print(max(f2_inds))

425006
424929


In [91]:
folds_list=[]
folds_list.append([f1_inds,f2_inds])
folds_list.append([f2_inds,f1_inds])

In [92]:
print(len(f1_inds))
print(len(f2_inds))

213752
211255


In [93]:
group = df.year.tolist()
target='VHI'
y = df[target]
x = df.drop(columns=[target])

In [94]:
gkf = GroupKFold(n_splits=2)
for train_index, test_index in gkf.split(x,groups=group):
  print('train: ',train_index, 'test: ', test_index)
  xtrain = x.iloc[train_index,:]
  ytrain = y[train_index]
  xtest = x.iloc[test_index,:]
  ytest = y[test_index]

train:  [    21     22     23 ... 577662 577663 577664] test:  [     0      1      2 ... 577684 577685 577686]
train:  [     0      1      2 ... 577684 577685 577686] test:  [    21     22     23 ... 577662 577663 577664]


In [95]:
print(xtrain['year'].unique())
print(xtest['year'].unique())

[2003 2004 2006 2008 2010 2014 2016 2018 2020 2021]
[2005 2007 2009 2011 2012 2013 2015 2017 2019]


In [96]:
#define estimator
def create_ff_model(neurons=500, dropout_rate=0, activation='relu', init_mode='RandomNormal', learn_rate=0.001, optimizer='RMSprop'):
    model=Sequential()
    model.add(Dense(neurons,input_shape=(16,),kernel_initializer=init_mode, activation='linear'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(neurons,kernel_initializer=init_mode, activation=activation, kernel_regularizer='l2'))
    #model.add(Dense(neurons,kernel_initializer=init_mode, activation=activation))
    #model.add(Dense(neurons,kernel_initializer=init_mode, activation=activation))
    #model.add(Dense(neurons,kernel_initializer=init_mode, activation=activation))
    model.add(Dense(1, activation='linear'))
    #compile
    model.compile(loss='mae', optimizer=optimizer)

    return model

In [97]:
# use grid search cv
# optimize number of estimators
    # Define hyperparameter space
batch_size = [int(x) for x in np.linspace(start = 5, stop = 100, num = 5)]
epochs = np.array((5,10,30))
optimizer = ['Adam', 'RMSprop', 'Nadam']
dropout_rate = [int(x) for x in np.linspace(start = 0.1, stop = 1, num = 5)]
#neurons = [int(x) for x in np.linspace(start = 5, stop = 1000, num = 50)]
activation = ['relu', 'selu', 'elu']
learn_rate = np.array((0.0001, 0.01, 0.1))
init_mode = ['RandomNormal', 'he_normal', 'VarianceScaling', 'TruncatedNormal']
neurons = np.array((25, 50, 250))
dropout_rate = np.array((0.0, 0.2, 0.4))
# can add settings to grid then add for gridsearchcv
param_grid = {'neurons':neurons,'epochs':epochs}

model = KerasRegressor(build_fn=create_ff_model, batch_size=10, epochs =1)

grid_cv = GridSearchCV(model,param_grid,cv=folds_list,scoring='r2',verbose=3)

<ipython-input-97-2fc2fa1a1dbb>:17: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasRegressor(build_fn=create_ff_model, batch_size=10, epochs =1)


In [98]:
print(train_df)

           lon     lat      vhi-1   precip-1      rzsm-1  spei1-1m  spei2-1m  \
0      -53.375 -32.125  70.414172  259.20056   98.021910  2.192168  1.313241   
1      -53.375 -32.125  79.323750  146.58607   98.162980  1.228278  2.453098   
2      -53.375 -32.125  77.111895  184.82845   97.594270  1.472914  1.588110   
3      -53.375 -32.125  64.150561  108.86273   97.605330  1.680747  1.751371   
4      -53.375 -32.125  54.270359  145.90760  100.000000  1.641170  1.900034   
...        ...     ...        ...        ...         ...       ...       ...   
577682 -47.625  -0.875  41.102345  170.38158   66.929490  0.324701  0.572972   
577683 -47.625  -0.875  49.922513  294.35620   80.138200  1.130223  0.881931   
577684 -47.625  -0.875  52.101102  125.48111   88.888840  2.664571  2.250521   
577685 -47.625  -0.875  46.753572   24.13338   77.834076  2.523025  3.002636   
577686 -47.625  -0.875  48.627606   71.58264   69.860950  1.286047  2.275886   

        spei3-1m   spi01-1   spi02-1   

In [99]:
trainx = train_df.drop(columns=['VHI'])
trainy = train_df['VHI']
# scale inputs
scaler = StandardScaler()
trainx_sc = scaler.fit_transform(trainx)

In [100]:
print(trainx.shape)

(425007, 16)


In [101]:
print(len(trainy))
print(len(trainx_sc))

425007
425007


In [102]:
print(trainy)

0         79.323750
1         77.111895
2         64.150561
3         54.270359
4         59.200781
            ...    
577682    49.922513
577683    52.101102
577684    46.753572
577685    48.627606
577686    47.450658
Name: VHI, Length: 425007, dtype: float64


In [103]:
grid_result = grid_cv.fit(trainx_sc,trainy)

Fitting 2 folds for each of 9 candidates, totalling 18 fits
Epoch 1/5
21376/21376 [==============================] - 35s 2ms/step - loss: 7.3882
Epoch 2/5
21376/21376 [==============================] - 34s 2ms/step - loss: 6.6404
Epoch 3/5
21376/21376 [==============================] - 36s 2ms/step - loss: 6.6130
Epoch 4/5
21376/21376 [==============================] - 35s 2ms/step - loss: 6.6005
Epoch 5/5
21126/21126 [==============================] - 26s 1ms/step
[CV 1/2] END ..............epochs=5, neurons=25;, score=0.592 total time= 3.9min
Epoch 1/5
21126/21126 [==============================] - 34s 2ms/step - loss: 6.9558
Epoch 2/5
21126/21126 [==============================] - 33s 2ms/step - loss: 6.1370
Epoch 3/5
21126/21126 [==============================] - 33s 2ms/step - loss: 6.1154
Epoch 4/5
21126/21126 [==============================] - 32s 2ms/step - loss: 6.1016
Epoch 5/5
21376/21376 [==============================] - 25s 1ms/step
[CV 2/2] END ..............epochs=5, ne

In [104]:
# produce plot of model fit average r2 scores against number of estimators
print(grid_result.best_score_)
print(grid_result.best_params_)

0.5940418313992497
{'epochs': 30, 'neurons': 25}


In [113]:
def create_ff_model(neurons=25, dropout_rate=0, activation='relu', init_mode='RandomNormal', learn_rate=0.001, optimizer='RMSprop'):
    model=Sequential()
    model.add(Dense(neurons,input_shape=(16,),kernel_initializer=init_mode, activation='linear'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(neurons,kernel_initializer=init_mode, activation=activation, kernel_regularizer='l2'))
    #model.add(Dense(neurons,kernel_initializer=init_mode, activation=activation))
    #model.add(Dense(neurons,kernel_initializer=init_mode, activation=activation))
    #model.add(Dense(neurons,kernel_initializer=init_mode, activation=activation))
    model.add(Dense(1, activation='linear'))
    #compile
    model.compile(loss='mae', optimizer=optimizer)

    return model

In [114]:
# test model with best params on hold out test dataset
model = KerasRegressor(build_fn=create_ff_model, batch_size=10, epochs =30)
model.fit(trainx_sc,trainy)

Epoch 1/30


<ipython-input-114-3b3685e733cc>:2: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasRegressor(build_fn=create_ff_model, batch_size=10, epochs =30)


42501/42501 [==============================] - 65s 2ms/step - loss: 6.9447
Epoch 2/30
42501/42501 [==============================] - 65s 2ms/step - loss: 6.3562
Epoch 3/30
42501/42501 [==============================] - 65s 2ms/step - loss: 6.3085
Epoch 4/30
42501/42501 [==============================] - 65s 2ms/step - loss: 6.2732
Epoch 5/30
42501/42501 [==============================] - 65s 2ms/step - loss: 6.2524
Epoch 6/30
42501/42501 [==============================] - 65s 2ms/step - loss: 6.2443
Epoch 7/30
42501/42501 [==============================] - 63s 1ms/step - loss: 6.2379
Epoch 8/30
42501/42501 [==============================] - 63s 1ms/step - loss: 6.1157
Epoch 9/30
42501/42501 [==============================] - 62s 1ms/step - loss: 6.0740
Epoch 10/30
42501/42501 [==============================] - 62s 1ms/step - loss: 6.0593
Epoch 11/30
42501/42501 [==============================] - 63s 1ms/step - loss: 6.0543
Epoch 12/30
42501/42501 [==============================] - 62s 

In [115]:
testx = test_df.drop(columns=['VHI'])
testy = test_df['VHI']
testx_sc = scaler.transform(testx)

In [116]:
predictions = model.predict(testx_sc)

15268/15268 [==============================] - 17s 1ms/step


In [117]:
print(r2_score(testy,predictions))
test_r2_score = r2_score(testy,predictions)

0.6036359923676962


In [118]:
# Get line plot of estimators vs R2 performance
# save model outputs to use in plotting script
print(len(predictions))
print(predictions.shape)
print(testx)

152680
(152680,)
           lon     lat      vhi-1    precip-1     rzsm-1  spei1-1m  spei2-1m  \
32     -53.375 -32.125  33.460501   31.077044  43.486305 -1.947763 -0.260165   
33     -53.375 -32.125  53.849729  115.865265  20.373812  0.641211 -0.586411   
34     -53.375 -32.125  62.237937   62.560310  22.396515 -1.185511 -0.421650   
35     -53.375 -32.125  60.625543  104.338350  22.017498 -0.140137 -0.993312   
36     -53.375 -32.125  64.313523   67.846820  23.099638 -0.963393 -0.731726   
...        ...     ...        ...         ...        ...       ...       ...   
577660 -47.625  -0.875  40.887218  273.721040  69.872240  0.256454 -0.125266   
577661 -47.625  -0.875  40.887218  138.636630  49.382520 -0.555288 -0.168773   
577662 -47.625  -0.875  50.892596  132.648000  52.529457  0.057421 -0.337412   
577663 -47.625  -0.875  43.292766   18.556600  48.998238  0.275302  0.111811   
577664 -47.625  -0.875  46.295286   36.166164  45.469177  1.248383  0.872078   

        spei3-1m   spi

In [119]:
# bring lat, lon, month and year together
results_df = pd.DataFrame({'lat':testx['lat'],'lon':testx['lon'],'month':testx['month'],'year':testx['year'],'predictions':predictions,'testy':testy})

In [120]:
results_df.to_csv('/content/drive/MyDrive/model_outs/ANN_out_df_optimized.csv',index=False)